## Test Retrieval Strategies

This is a notebook that explores, in detail, different chunking and retrieval strategies. 

## Formatting code

The next cell is a function that will wrap the output of the code cells in a way that makes it easier to read.  It is a workaround for the fact that Jupyter notebooks in Cursor do not wrap the same way that they do in Colab.

In [1]:
from dean_utils.format import set_output_wrapping
set_output_wrapping()

## API Key Setup
---

In [2]:
from dean_utils.defaults import set_api_key
# Example usage:
# set_api_key("OPENAI_API_KEY")
# default_llm = ChatOpenAI(model="gpt-4o", api_key=set_api_key("OPENAI_API_KEY"))

## Let's explore different pdf loaders and settings

##### (Here I am just using the TITRE-protocol.pdf file for testing.  Later I will need a file input routine so that I can distribute this notebook to others without TITRE.)
---

### PyPDFLoader and PyMuPDFLoader both have same syntax.


---
#### Using default mode = "page"
---

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the protocol and cut it up into chunks
loader = PyPDFLoader("TITRE-protocol.pdf", mode="page")
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=50)
splits = text_splitter.split_documents(pages)
print(f"Page mode loaded the document in individual pages, {len(pages)} total.")
print(f"Then the text splitter worked on each page, resulting in {len(splits)} chunks.")

Page mode loaded the document in individual pages, 51 total.
Then the text splitter worked on each page, resulting in 73 chunks.


---
### mode = "single" should combine all the text into one

---

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

# Load the protocol and cut it up into chunks
loader = PyMuPDFLoader("TITRE-protocol.pdf", mode="single")
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=50)
splits = text_splitter.split_documents(pages)
print(f"Page mode loaded the document as {len(pages)} page of text.")
print(f"Then the text splitter worked on that text, resulting in {len(splits)} chunks.")

Page mode loaded the document as 1 page of text.
Then the text splitter worked on that text, resulting in 43 chunks.


PyMuPDFLoader automatically provides more metadata fields than PyPDFLoader so for the rest of this notebook we will just use PyMuPDFLoader.  Let's look at the default metadata using pprint to properly display the fields.

In [5]:
import pprint
pprint.pp(pages[0].metadata)

{'producer': 'Microsoft® Word 2016',
 'creator': 'Microsoft® Word 2016',
 'creationdate': '2024-01-05T10:56:40-05:00',
 'source': 'TITRE-protocol.pdf',
 'file_path': 'TITRE-protocol.pdf',
 'total_pages': 51,
 'format': 'PDF 1.5',
 'title': '',
 'author': 'Thiagarajan, Ravi',
 'subject': '',
 'keywords': '',
 'moddate': '2024-01-05T10:56:40-05:00',
 'trapped': '',
 'modDate': "D:20240105105640-05'00'",
 'creationDate': "D:20240105105640-05'00'"}


### Compare Modes of Loading for PyMuPDFLoader

Note that these modes also apply to PyPDFLoader.  Our choice between the loaders is arbitrary.  In the previous cells we saw that loading with page or single mode results in different numbers of chunks.  The next cell illustrates this.  Remember that a chunk size of 3000 is clearly MORE than the size of some *small pages* in the original PDF document.  


In [6]:
loader = PyMuPDFLoader("TITRE-protocol.pdf",
                       mode="page",)
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=50)
splits = text_splitter.split_documents(pages)
print(f"Page mode number of pages: {len(pages)}, number of chunks: {len(splits)}")
print(f"Size of the first six pages: {', '.join(str(len(pages[i].page_content)) for i in range(6))}")
print(f"Length of first eight chunks: {', '.join(str(len(splits[i].page_content)) for i in range(8))}")

loader = PyMuPDFLoader("TITRE-protocol.pdf",
                       mode="single",)
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=50)
splits = text_splitter.split_documents(pages)
print(f"Single mode number of pages: {len(pages)}, number of chunks: {len(splits)}")
print(f"Length of first five chunks: {', '.join(str(len(splits[i].page_content)) for i in range(5))}")


Page mode number of pages: 51, number of chunks: 73
Size of the first six pages: 1352, 4893, 3836, 1274, 1048, 1612
Length of first eight chunks: 1352, 2921, 1970, 2910, 924, 1274, 1048, 1612
Single mode number of pages: 1, number of chunks: 43
Length of first five chunks: 2927, 2960, 2930, 2993, 2981


###  Important Conclusion from above

The default mode for PyMuPDFLoader is "page".  What happens is that if the chunk size is larger than the page size, the loader will load the entire page as a single chunk.  The first page of this document had 1352 characters, and the first chunk also had 1352.  The next page had 4893 characters and was split into two chunks;  the first of these was close to the 3000 chunk size setting and the second was the remaining 1970 characters. 

This is not usually the desired behavior, because our chunk overlap strategy will not work *between chunks from different pages*.  So we need to use the "single" mode, which loads the entire document as a single chunk.  Then we will get chunks that do have overlap between them (*if that was the behavior we wanted*.)


## Now let's set up experiments

We are going to chunk our test document into different chunk sizes with different overlaps, store each of those strategies in a separate Qdrant collection, and then create rag chains that will use each of those collections.  Then we will ask questions of the chain and observe which strategies completely fail.

---


In [7]:
from dean_utils.getVectorstore import getVectorstore

def create_vector_store(file_path, embedding_model, chunk_size, chunk_overlap):
    loader = PyMuPDFLoader(file_path, mode="single",)
    pages = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(pages)
    collection_name = f"ChunkSize_{chunk_size}_Overlap_{chunk_overlap}"
    return getVectorstore(document=chunks,file_path=file_path, embedding_model=embedding_model, collection_name=collection_name)

In the upcoming code cell, you will see that I imported two embedding models so I could compare their performance.  The BGE model was vastly inferior to the OpenAI text-embedding-3-small model, and so the code below uses the commercial model.

In [8]:
from dean_utils.defaults import bge_embedding_model, te3_embedding_model
embedding_model = te3_embedding_model
file_path="TITRE-protocol.pdf"
vectorStore1 = create_vector_store(file_path, embedding_model, 1000, 100)
vectorStore2 = create_vector_store(file_path, embedding_model, 1000, 200)
vectorStore3 = create_vector_store(file_path, embedding_model, 1000, 300)
vectorStore4 = create_vector_store(file_path, embedding_model, 1000, 400)
vectorStore5 = create_vector_store(file_path, embedding_model, 1000, 500)
vectorStore6 = create_vector_store(file_path, embedding_model, 2000, 100)
vectorStore7 = create_vector_store(file_path, embedding_model, 2000, 200)
vectorStore8 = create_vector_store(file_path, embedding_model, 2000, 300)
vectorStore9 = create_vector_store(file_path, embedding_model, 2000, 400)
vectorStore10 = create_vector_store(file_path, embedding_model, 2000, 500)

There is also a parameter k that determines how many chunks are returned, and we can also test changes in k to see the effects.

When k = 3, the later queries about inclusion criteria fail in the majority of instances.  When k = 5, the queries succeed in almost all cases.  When k = 8, there are still a couple of failures.  When I increase it to 20, 100% of the queries succeed using GPT-4o as the model.

In [14]:
k = 10
retrievers = [
    vectorStore1.as_retriever(search_kwargs={"k": k}),
    vectorStore2.as_retriever(search_kwargs={"k": k}),
    vectorStore3.as_retriever(search_kwargs={"k": k}),
    vectorStore4.as_retriever(search_kwargs={"k": k}),
    vectorStore5.as_retriever(search_kwargs={"k": k}),
    vectorStore6.as_retriever(search_kwargs={"k": k}),
    vectorStore7.as_retriever(search_kwargs={"k": k}),
    vectorStore8.as_retriever(search_kwargs={"k": k}),
    vectorStore9.as_retriever(search_kwargs={"k": k}),
    vectorStore10.as_retriever(search_kwargs={"k": k})
]

In [15]:
from operator import itemgetter
from dean_utils.prompts import rag_prompt_template
from dean_utils.defaults import gpt4o_llm
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)
llm = gpt4o_llm

def create_rag_chains_with_metadata(retrievers, rag_prompt, llm, chunk_sizes, chunk_overlaps):
    """
    Create RAG chains with metadata about chunk sizes and overlaps.
    """
    rag_chains = {}
    for i, (retriever, chunk_size, chunk_overlap) in enumerate(zip(retrievers, chunk_sizes, chunk_overlaps), 1):
        chain_name = f"rag_chain_{i}_chunk{chunk_size}_overlap{chunk_overlap}"
        rag_chains[chain_name] = {
            "chain": (
                {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
                | rag_prompt | llm | StrOutputParser()
            ),
            "metadata": {
                "chunk_size": chunk_size,
                "chunk_overlap": chunk_overlap
            }
        }
    return rag_chains

# Create chains with metadata
chunk_sizes = [1000] * 5 + [2000] * 5  # First 5 are 1000, last 5 are 2000
chunk_overlaps = [100, 200, 300, 400, 500] * 2  # Repeat for both chunk sizes
rag_chains = create_rag_chains_with_metadata(retrievers, rag_prompt, llm, chunk_sizes, chunk_overlaps)

# Example usage:
# response = rag_chains["rag_chain_1_chunk1000_overlap100"]["chain"].invoke({"question": "What is the title?"})

In [16]:

print(rag_chains["rag_chain_1_chunk1000_overlap100"]["chain"].invoke({"question": "What is the title of the protocol?"}))
print(rag_chains["rag_chain_2_chunk1000_overlap200"]["chain"].invoke({"question": "What is the title of the protocol?"}))
print(rag_chains["rag_chain_3_chunk1000_overlap300"]["chain"].invoke({"question": "What is the title of the protocol?"}))
print(rag_chains["rag_chain_4_chunk1000_overlap400"]["chain"].invoke({"question": "What is the title of the protocol?"}))
print(rag_chains["rag_chain_5_chunk1000_overlap500"]["chain"].invoke({"question": "What is the title of the protocol?"}))
print(rag_chains["rag_chain_6_chunk2000_overlap100"]["chain"].invoke({"question": "What is the title of the protocol?"}))
print(rag_chains["rag_chain_7_chunk2000_overlap200"]["chain"].invoke({"question": "What is the title of the protocol?"}))
print(rag_chains["rag_chain_8_chunk2000_overlap300"]["chain"].invoke({"question": "What is the title of the protocol?"}))
print(rag_chains["rag_chain_9_chunk2000_overlap400"]["chain"].invoke({"question": "What is the title of the protocol?"}))
print(rag_chains["rag_chain_10_chunk2000_overlap500"]["chain"].invoke({"question": "What is the title of the protocol?"}))


The title of the protocol is "TITRE: Trial of Indication-based Transfusion of Red Blood Cells in ECMO."
The title of the protocol is "TITRE: Trial of Indication-based Transfusion of Red Blood Cells in ECMO."
The title of the protocol is "TITRE: Trial of Indication-based Transfusion of Red Blood Cells in ECMO."
The title of the protocol is "TITRE: Trial of Indication-based Transfusion of Red Blood Cells in ECMO."
The title of the protocol is "TITRE: Trial of Indication-based Transfusion of Red Blood Cells in ECMO."
The title of the protocol is "TITRE: Trial of Indication-based Transfusion of Red Blood Cells in ECMO."
The title of the protocol is "TITRE: Trial of Indication-based Transfusion of Red Blood Cells in ECMO."
The title of the protocol is "TITRE: Trial of Indication-based Transfusion of Red Blood Cells in ECMO."
The title of the protocol is "TITRE: Trial of Indication-based Transfusion of Red Blood Cells in ECMO."
The title of the protocol is "TITRE: Trial of Indication-based T

In [17]:
print("=" * 50)
print(rag_chains["rag_chain_1_chunk1000_overlap100"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_2_chunk1000_overlap200"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_3_chunk1000_overlap300"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_4_chunk1000_overlap400"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_5_chunk1000_overlap500"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_6_chunk2000_overlap100"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_7_chunk2000_overlap200"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_8_chunk2000_overlap300"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_9_chunk2000_overlap400"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_10_chunk2000_overlap500"]["chain"].invoke({"question": "What are the inclusion criteria of the protocol?"}))


The inclusion criteria for the protocol are as follows:

1. Age less than 6 years at ECMO cannulation.
2. Veno-arterial (VA) mode of ECMO.
3. First ECMO run during the index hospitalization.
The inclusion criteria for the protocol are as follows:

1. Children supported with ECMO for any indication.
2. Age less than 6 years at ECMO cannulation.
3. Veno-arterial (VA) mode of ECMO.
4. First ECMO run during the index hospital admission.
The inclusion criteria for the protocol are as follows:

1. Children supported with ECMO for any indication.
2. Age less than 6 years at ECMO cannulation.
3. Veno-arterial (VA) mode of ECMO.
4. First ECMO run during the index hospital admission.
The inclusion criteria of the protocol are that all must be met, but the specific criteria are not provided in the context.
The inclusion criteria of the protocol are as follows:

1. Age less than 6 years at ECMO cannulation.
2. Veno-arterial (VA) mode of ECMO.
3. First ECMO run during the index hospitalization.
The

In [18]:
print("=" * 50)
print(rag_chains["rag_chain_1_chunk1000_overlap100"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_2_chunk1000_overlap200"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_3_chunk1000_overlap300"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_4_chunk1000_overlap400"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_5_chunk1000_overlap500"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_6_chunk2000_overlap100"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_7_chunk2000_overlap200"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_8_chunk2000_overlap300"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_9_chunk2000_overlap400"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_10_chunk2000_overlap500"]["chain"].invoke({"question": "What are the exclusion criteria of the protocol?"}))


The exclusion criteria of the protocol are as follows:

1. Gestationally-corrected age < 37 weeks.
2. Veno-venous (VV) mode of ECMO.
3. Patients initially started on VV-ECMO and then transitioned to VA ECMO.
4. ECMO used for procedural support (ECMO deployed and decannulated in procedural area with no ICU ECMO care) or ECMO duration expected to be < 24 hours.
5. Limitation of care in place or being discussed.
6. Congenital bleeding disorders.
7. Hemoglobinopathies.
8. Primary residence outside the country of enrollment.
9. Concurrent participation in a separate interventional trial that has the potential to impact neurodevelopmental status of the patient.
10. Patients cannulated for ECMO at a non-trial center and transferred to a trial site.
11. Randomization not possible within 36 hours following ECMO cannulation (e.g., due to staffing or delays related to communication with participant family).
The exclusion criteria of the protocol are as follows:

1. Gestationally-corrected age < 3

In [19]:
print("=" * 50)
print(rag_chains["rag_chain_1_chunk1000_overlap100"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_2_chunk1000_overlap200"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_3_chunk1000_overlap300"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_4_chunk1000_overlap400"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_5_chunk1000_overlap500"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_6_chunk2000_overlap100"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_7_chunk2000_overlap200"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_8_chunk2000_overlap300"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_9_chunk2000_overlap400"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))
print("=" * 50)
print(rag_chains["rag_chain_10_chunk2000_overlap500"]["chain"].invoke({"question": "What are the 4th and 7th exclusion criteria of the protocol?"}))


The 4th exclusion criterion is: "ECMO used for procedural support (ECMO deployed and decannulated in procedural area with no ICU ECMO care) or ECMO duration expected to be < 24 h."

The 7th exclusion criterion is: "Hemoglobinopathies."
The 4th exclusion criterion is: "ECMO used for procedural support (ECMO deployed and decannulated in procedural area with no ICU ECMO care) or ECMO duration expected to be < 24 h."

The 7th exclusion criterion is: "Hemoglobinopathies."
The 4th exclusion criterion is: "ECMO used for procedural support (ECMO deployed and decannulated in procedural area with no ICU ECMO care) or ECMO duration expected to be < 24 h."

The 7th exclusion criterion is: "Hemoglobinopathies."
The 4th exclusion criterion is: "ECMO used for procedural support (ECMO deployed and decannulated in procedural area with no ICU ECMO care) or ECMO duration expected to be < 24 h."

The 7th exclusion criterion is: "Hemoglobinopathies."
The 4th exclusion criterion is: "ECMO used for procedura

### Conclusions to this point
---

Chunk size and overlap do certainly affect the performance of retrieval, but this is much more pronounced when using the BGE model.  This is probably because it is a 768 embedding model.  

Increasing k did not completely solve the issue when using BGE.  Lesson to me is to use the richest embedding model we can afford to use.

The next experiment is to swap out the model and hit a local model on my laptop.  More to come!

---